Let's import all our necessary libraries

In [2]:
import requests
import pandas as pd
import sqlite3
from sqlalchemy.types import * 
import sqlalchemy
import re

I'd like to look for all the studies whose titles reference Prostate Cancer. To limit the number of pages we have to request/parse through, let's set the pageSize to 100. 

In [11]:
api_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "query.titles": "Prostate Cancer",
    "pageSize": 100
}

Let's create some empty lists for appending to dataframes while we request the data we are interested in. Additionally, I'll create a dictionary of state abbreviations to convert the pre-existing state names to abbreviations for later visualization with Plotly

In [ ]:
main_data_list = []
phases_df_list = []
interventions_df_list = []
organizations_df_list = []
collaborators_df_list = []
conditions_df_list = []
mesh_conditions_df_list = []
mesh_interventions_df_list = []
serious_organs_df_list = []
other_organs_df_list = []
locations_df_list = []
outcome_title_df_list = []
outcomes_df_list = []

state_abbreviations = {
    "alaska": "AK",
    "alabama": "AL",
    "arkansas": "AR",
    "arizona": "AZ",
    "california": "CA",
    "colorado": "CO",
    "connecticut": "CT",
    "delaware": "DE",
    "florida": "FL",
    "georgia": "GA",
    "hawaii": 'HI',
    "iowa": "IA",
    "idaho": "IA",
    "illinois": "IL",
    "indiana": "IN",
    "kansas": "KS",
    "kentucky": "KY",
    "louisiana": "LA",
    "massachusetts": "MA",
    "maryland": "MD",
    "maine": "ME",
    "michigan": "MI",
    "minnesota": "MN",
    "missouri": "MO",
    "mississippi": "MS",
    "montana": "MT",
    "north carolina": "NC",
    "north dakota": "ND",
    "nebraska": "NE",
    "new hampshire": "NH",
    "new jersey": "NJ",
    "new mexico": "NM",
    "nevada": "NV",
    "new york": "NY",
    "ohio": "OH",
    "oklahoma": "OK",
    "oregon": "OR",
    "pennsylvania": "PA",
    "rhode island": "RI",
    "south carolina": "SC",
    "south dakota": "SD",
    "tennessee": "TN",
    "texas": "TX",
    "utah": 'UT',
    "virginia": "VA",
    "vermont": "VT",
    "washington": "WA",
    "wisconsin": "WI",
    "west virginia": "WV",
    "wyoming": "WY",
    "district of columbia": "DC",
    "american samoa": "AS",
    "guam gu": "GU",
    "northern mariana islands": "MP",
    "puerto rico pr": "PR",
    "us virgin islands": "VI"
}

Main request block. Extracts data and appends to dataframes while getting a successful status code. If an error code is returned, or if no nextPage token is found, break. We'll also do some preliminary data cleaning and normalization by converting all string data to lowercase. Additionally, let's convert all dates to a common format, e.g. yyyy-mm-dd. Finally, we convert any states found in our previously defined dictionary to their abbreviation.

In [ ]:
while True:
    print('Fetching data from url: ', api_url + '?' + '&'.join([f"{k}={v}" for k, v in params.items()]))

    response = requests.get(api_url, params=params, timeout=2.5)
    
    if response.status_code == 200:
        data = response.json()
        studies = data.get('studies', [])

        
        for study in studies:
            nctID = study['protocolSection']['identificationModule'].get('nctId', 'NULL')
            briefTitle = study['protocolSection']['identificationModule'].get('briefTitle', 'NULL')
            acronym = study['protocolSection']['identificationModule'].get('acronym', 'NULL')
            orgName = study['protocolSection']['identificationModule'].get('organization', {}).get('fullName', 'NULL')
            orgType = study['protocolSection']['identificationModule'].get('organization', {}).get('class', 'NULL')

            overallStatus = study['protocolSection']['statusModule'].get('overallStatus', 'NULL')




            startDate = study['protocolSection']['statusModule'].get('startDateStruct', {}).get('date', '1900-00-00')
            if re.match(r'^[0-9]{4}\-[0-9]{2}\-[0-9]{2}$',startDate):
                pass
            else:
                startDate = startDate + '-01'
            completionDate = study['protocolSection']['statusModule'].get('completionDateStruct', {}).get('date', '1900-00-00')
            if re.match(r'^[0-9]{4}\-[0-9]{2}\-[0-9]{2}$',completionDate):
                pass
            else:
                completionDate = completionDate + '-01'
            lastUpdate = study['protocolSection']['statusModule'].get('lastUpdatePostDateStruct', {}).get('date', '1900-00-00')
            if re.match(r'^[0-9]{4}\-[0-9]{2}\-[0-9]{2}$',lastUpdate):
                pass
            else:
                lastUpdate = lastUpdate + '-01'



            investigatorFullName = study['protocolSection']['sponsorCollaboratorsModule'].get('responsibleParty', {}).get('investigatorFullName', 'NULL')
            sponsorName = study['protocolSection']['sponsorCollaboratorsModule'].get('leadSponsor', {}).get('name', 'NULL')
            all_collaborators = study['protocolSection']['sponsorCollaboratorsModule'].get('collaborators', [])
            collaborators = '$'.join([collaborator.get('name', 'NULL') for collaborator in all_collaborators]) if all_collaborators else "NULL"


            isFdaRegulatedDrug = study['protocolSection'].get('oversightModule', {}).get('isFdaRegulatedDrug', 'NULL')

            all_conditions = study['protocolSection']['conditionsModule'].get('conditions', 'NULL')
            conditions = '$'.join([condition for condition in all_conditions]) if all_conditions else "NULL"
            numConditions = study['protocolSection']['conditionsModule'].get('numConditions', 'NULL')

            studyType = study['protocolSection']['designModule'].get('studyType', 'NULL')
            
            all_interventions = study['protocolSection'].get('armsInterventionsModule', {}).get('interventions', [])
            interventionName = '$'.join([intervention.get('name', 'NULL') for intervention in all_interventions]) if all_interventions else "NULL"
            interventionType = '$'.join([intertype.get('type', 'NULL') for intertype in all_interventions]) if all_interventions else "NULL"
            
            numInterventions = study['protocolSection'].get('armsInterventionsModule', {}).get('numInterventions', 'NULL')
            interventionalModel = study['protocolSection']['designModule'].get('designInfo', {}).get('interventionModel', 'NULL')


            studyPhase = '$'.join(study['protocolSection']['designModule'].get('phases', ['NULL']))
            primaryPurpose = study['protocolSection']['designModule'].get('designInfo', {}).get('primaryPurpose', 'NULL')
            designAllocation = study['protocolSection']['designModule'].get('designInfo', {}).get('allocation', 'NULL')
            enrollmentcount = study['protocolSection']['designModule'].get('enrollmentInfo', {}).get('count', 'NULL')
            enrollmenttype = study['protocolSection']['designModule'].get('enrollmentInfo', {}).get('type', 'NULL')
            
            

            genderBased = study['protocolSection']['eligibilityModule'].get('genderBased', 'NULL')
            sex = study['protocolSection']['eligibilityModule'].get('sex', 'NULL')
            minAge = study['protocolSection']['eligibilityModule'].get('minimumAge', 'NULL')
            maxAge = study['protocolSection']['eligibilityModule'].get('maximumAge', 'NULL')

            all_locations = study['protocolSection'].get('contactsLocationsModule', {}).get('locations', [])
            locations = '$'.join([f"{location.get('city', 'NULL CITY')} - {location.get('state', 'NULL STATE')} - {location.get('country', 'NULL COUNTRY')}" for location in all_locations]) if all_locations else "NULL"



            hasResults = study.get('hasResults', 'NULL')


            frequencyThreshold = study.get('resultsSection', {}).get('adverseEventsModule', {}).get('frequencyThreshold', 'NULL')

            all_adverseEvents = study.get('resultsSection', {}).get('adverseEventsModule', {}).get('eventGroups', [])
            deathsNumAffected = [event.get('deathsNumAffected', 'NULL') for event in all_adverseEvents] if all_adverseEvents else "NULL"
            if 'NULL' in deathsNumAffected:
                deathsNumAffected = 'NULL'
            else:
                deathsNumAffected = sum([int(deaths) for deaths in deathsNumAffected])
            

            seriousNumAffected = [event.get('seriousNumAffected', 'NULL') for event in all_adverseEvents] if all_adverseEvents else "NULL"
            if 'NULL' in seriousNumAffected:
                pass
            else:
                seriousNumAffected = sum([int(serious) for serious in seriousNumAffected])
            

            otherNumAffected = [event.get('otherNumAffected', 'NULL') for event in all_adverseEvents] if all_adverseEvents else "NULL"
            if 'NULL' in otherNumAffected:
                pass
            else:
                otherNumAffected = sum([int(other) for other in otherNumAffected])

            
            all_seriousEvents = study.get('resultsSection', {}).get('adverseEventsModule', {}).get('seriousEvents', [])
            seriousorganSystem = '$'.join([event.get('organSystem', 'NULL') for event in all_seriousEvents]) if all_seriousEvents else "NULL"
            
            all_otherEvents = study.get('resultsSection', {}).get('adverseEventsModule', {}).get('otherEvents', [])
            otherorganSystem = '$'.join([event.get('organSystem', 'NULL') for event in all_otherEvents]) if all_otherEvents else "NULL"

            all_meshids = study.get('derivedSection', {}).get('conditionBrowseModule', {}).get('meshes', [])
            meshID = '$'.join([mesh.get('id', 'NULL') for mesh in all_meshids]) if all_meshids else "NULL"
            
            all_meshinters = study.get('derivedSection', {}).get('interventionBrowseModule', {}).get('meshes', [])
            meshIntervention = '$'.join([meshint.get('term', 'NULL') for meshint in all_meshinters]) if all_meshinters else "NULL"



            main_data_list.append({
                "nctID": nctID,
                "BriefTitle": briefTitle,
                "Acronym": acronym,
                "OrgName": orgName,
                "Investigator": investigatorFullName,
                "Sponsor": sponsorName,
                "OverallStatus": overallStatus,
                "Interventional Model": interventionalModel,
                "FDARegulated": isFdaRegulatedDrug,
                "GenderBased": genderBased,
                "Sex": sex,
                "MinAge": minAge,
                "MaxAge": maxAge,
                "PrimaryPurpose": primaryPurpose,
                'Allocation': designAllocation,
                "EnrollmentCount": enrollmentcount,
                "EnrollmentType": enrollmenttype,
                "FreqAdverseEventsThreshold": frequencyThreshold,
                "NumDeaths": deathsNumAffected,
                "NumAffectedSeriousEvent": seriousNumAffected,
                "NumAffectedOtherEvent": otherNumAffected,
                "HasResults": hasResults,
                "StudyStartDate": startDate,
                "StudyCompletionDate": completionDate,
                "LastUpdatePostDate": lastUpdate,
                "StudyType": studyType,
            })

            
            if '$' in studyPhase:
                for phase in studyPhase.split('$'):
                    phases_df_list.append({
                        "nctID": nctID,
                        "Phase": phase.strip()
                    })
            else:
                phases_df_list.append({
                    "nctID": nctID,
                    "Phase": studyPhase
                })

            if '$' in interventionName:
                for intervention, intertype in zip(interventionName.split('$'), interventionType.split('$')):
                    interventions_df_list.append({
                        "nctID": nctID,
                        "Intervention": intervention.strip(),
                        "InterventionType": intertype
                    })
            else:
                interventions_df_list.append({
                    "nctID": nctID,
                    "Intervention": interventionName,
                    "InterventionType": interventionType
                })


            organizations_df_list.append({
                "OrgName": orgName,
                "OrgType": orgType
            })

            if '$' in collaborators:
                for collaborator in collaborators.split('$'):
                    collaborators_df_list.append({
                        "nctID": nctID,
                        "Collaborator": collaborator.strip()
                    })
            else:
                collaborators_df_list.append({
                    "nctID": nctID,
                    "Collaborator": collaborators
                })

            if '$' in conditions:
                for condition in conditions.split('$'):
                    conditions_df_list.append({
                        "nctID": nctID,
                        "Condition": condition.strip()
                    })
            else:
                conditions_df_list.append({
                    "nctID": nctID,
                    "Condition": conditions
                })
            
            if '$' in meshID:
                for mid in meshID.split('$'):
                    mesh_conditions_df_list.append({
                        "nctID": nctID,
                        "MeshConditionID": mid.strip()
                    })
            else:
                mesh_conditions_df_list.append({
                    "nctID": nctID,
                    "MeshConditionID": meshID
                })
            
            if '$' in meshIntervention:
                for inter in meshIntervention.split('$'):
                    mesh_interventions_df_list.append({
                        "nctID": nctID,
                        "MeshIntervention": inter.strip()
                    })
            else:
                mesh_interventions_df_list.append({
                    "nctID": nctID,
                    "MeshIntervention": meshIntervention
                })

            auxiliary_list = []
            if '$' in seriousorganSystem:
                for seriousorgan in seriousorganSystem.split('$'):
                    if seriousorgan not in auxiliary_list:
                        auxiliary_list.append(seriousorgan)
                        serious_organs_df_list.append({
                            "nctID": nctID,
                            "OrganSystemSeriousEvent": seriousorgan.strip()
                        })
            else:
                serious_organs_df_list.append({
                    "nctID": nctID,
                    "OrganSystemSeriousEvent": seriousorganSystem
                })

            auxiliary_list = []
            if '$' in otherorganSystem:
                for otherorgan in otherorganSystem.split('$'):
                    if otherorgan not in auxiliary_list:
                        auxiliary_list.append(otherorgan)
                        other_organs_df_list.append({
                            "nctID": nctID,
                            "OrganSystemOtherEvent": otherorgan.strip()
                        })
            else:
                other_organs_df_list.append({
                    "nctID": nctID,
                    "OrganSystemOtherEvent": otherorganSystem
                })
            
            if '$' in locations:
                for location in locations.split('$'):
                    if '-' in location:
                        elements = location.split('-')
                        locations_df_list.append({
                            "nctID": nctID,
                            "City": elements[0].strip(),
                            "State": elements[1].strip(),
                            "Country": elements[2].strip()
                        })
                        
            else:
                if '-' in locations:
                    elements = locations.split('-')
                    locations_df_list.append({
                        "nctID": nctID,
                        "City": elements[0].strip(),
                        "State": elements[1].strip(),
                        "Country": elements[2].strip()
                    })

            
                    
        nextPageToken = data.get('nextPageToken')
        if nextPageToken:
            params['pageToken'] = nextPageToken  # Set the pageToken for the next request
        else:
            print('All done!')
            break  # Exit the loop if no nextPageToken is present
    else:
        print('GET request failed. Error code: ', response.status_code)

main_df = pd.DataFrame(main_data_list).astype(str).apply(lambda x: x.str.lower())
phases_df = pd.DataFrame(phases_df_list).astype(str).apply(lambda x: x.str.lower())
interventions_df = pd.DataFrame(interventions_df_list).astype(str).apply(lambda x: x.str.lower())
organizations_df = pd.DataFrame(organizations_df_list).astype(str).apply(lambda x: x.str.lower())
collaborators_df = pd.DataFrame(collaborators_df_list).astype(str).apply(lambda x: x.str.lower())
conditions_df = pd.DataFrame(conditions_df_list).astype(str).apply(lambda x: x.str.lower())
mesh_conditions_df = pd.DataFrame(mesh_conditions_df_list).astype(str).apply(lambda x: x.str.lower())
mesh_interventions_df = pd.DataFrame(mesh_interventions_df_list).astype(str).apply(lambda x: x.str.lower())
serious_organs_df = pd.DataFrame(serious_organs_df_list).astype(str).apply(lambda x: x.str.lower())
other_organs_df =  pd.DataFrame(other_organs_df_list).astype(str).apply(lambda x: x.str.lower())
locations_df = pd.DataFrame(locations_df_list).astype(str).apply(lambda x: x.str.lower())

for k, v in state_abbreviations.items():
    locations_df['State'] = locations_df['State'].replace(k, v)

Fetching data from url:  https://clinicaltrials.gov/api/v2/studies?query.titles=Prostate Cancer&pageSize=100
Fetching data from url:  https://clinicaltrials.gov/api/v2/studies?query.titles=Prostate Cancer&pageSize=100&pageToken=ZVNj7o2Elu8o3lpsBMv7oqr2mpOQJJxoZ_ag1fg
Fetching data from url:  https://clinicaltrials.gov/api/v2/studies?query.titles=Prostate Cancer&pageSize=100&pageToken=ZVNj7o2Elu8o3lpsBMv7oqr2mpOQJJxuY_Oh2fQW
Fetching data from url:  https://clinicaltrials.gov/api/v2/studies?query.titles=Prostate Cancer&pageSize=100&pageToken=ZVNj7o2Elu8o3lpsBMv7oqr2mpOQJJxtZvCn2PUW
Fetching data from url:  https://clinicaltrials.gov/api/v2/studies?query.titles=Prostate Cancer&pageSize=100&pageToken=ZVNj7o2Elu8o3lpsBMv7oqr2mpOQJJxtbPei3PEW
Fetching data from url:  https://clinicaltrials.gov/api/v2/studies?query.titles=Prostate Cancer&pageSize=100&pageToken=ZVNj7o2Elu8o3lpsBMv7oqr2mpOQJJxsY_im2_cW
Fetching data from url:  https://clinicaltrials.gov/api/v2/studies?query.titles=Prostate Can

Taking a quick look at our locations dataframe to ensure the state names were replaced correctly.

In [13]:
print(locations_df)

             nctID            City State        Country
0      nct02987829      birmingham    AL  united states
1      nct02987829      scottsdale    AZ  united states
2      nct02987829    santa monica    CA  united states
3      nct02987829        new york    NY  united states
4      nct02987829  salt lake city    UT  united states
...            ...             ...   ...            ...
65205  nct00629057   lawrenceville    NJ  united states
65206  nct00629057       charlotte    NC  united states
65207  nct00629057       nashville    TN  united states
65208  nct00629057          dallas    TX  united states
65209  nct00629057         mcallen    TX  united states

[65210 rows x 4 columns]


Let's look at counts of unique values in all our columns to see how many unique values we have, including the NULL values

In [12]:
for column in main_df:
    print(main_df[column].value_counts())
    print('-'*50)

nctID
nct00121212    1
nct04015102    1
nct00292045    1
nct00384839    1
nct06663007    1
              ..
nct05461261    1
nct00654186    1
nct03644303    1
nct04543903    1
nct01656161    1
Name: count, Length: 5318, dtype: int64
--------------------------------------------------
BriefTitle
radiation therapy in treating patients with prostate cancer                                                                    3
vaccine therapy in treating patients with metastatic prostate cancer                                                           3
monoclonal antibody therapy in treating patients with prostate cancer                                                          3
extension study investigating the long-term safety and tolerability of repeat doses of fe200486 in prostate cancer patients    2
antineoplaston therapy in treating patients with stage iii or stage iv prostate cancer                                         2
                                                            

Now, for storing the data in a local SQL database, we need to ensure all columns are of the correct type. We do this by creating dictionaries containing column names and the corresponding type.

In [ ]:
main_df_form = {
    "nctID": 'string',
    "BriefTitle": 'string',
    "Acronym": 'string',
    "OrgName": 'string',
    "Investigator": 'string',
    "Sponsor": 'string',
    "OverallStatus": 'string',
    "StudyStartDate": 'string',
    "InterventionalModel": 'string',
    "FDARegulated": 'integer',
    "GenderBased": 'integer',
    "Sex": 'string',
    "MinAge": 'integer',
    "MaxAge": 'integer',
    "PrimaryPurpose": 'string',
    'Allocation': 'string',
    "EnrollmentCount": 'string',
    "EnrollmentType": 'string',
    "FreqAdverseEventsThreshold": 'real',
    "NumDeaths": 'integer',
    "NumAffectedSeriousEvent": 'integer',
    "NumAffectedOtherEvent": 'integer',
    "HasResults": 'integer',
    "StudyCompletionDate": 'string',
    "LastUpdatePostDate": 'string',
    "StudyType": 'string',
    }

phases_df_form = {
    "nctID": 'string',
    "Phase": 'string'
}

intervention_df_form = {
    "nctID": 'string',
    "Intervention": 'string',
    "InterventionType": 'string'
}

organization_df_form = {
    "OrgName": 'string',
    "OrgType": 'string'
}

collaborator_df_form = {
    "nctID": 'string',
    "Collaborator": 'string'
}

conditions_df_form = {
    "nctID": 'string',
    "Condition": 'string'
}

mesh_conditions_df_form = {
    "nctID": 'string',
    "MeshConditionID": 'string'
}

mesh_interventions_df_form = {
    "nctID": 'string',
    "MeshIntervention": 'string'
}

serious_organs_df_form = {
    "nctID": 'string',
    "OrganSystemSeriousEvent": 'string'
}

other_organs_df_form = {
    "nctID": 'string',
    "OrganSystemOtherEvent": 'string'
}

locations_df_form = {
    "nctID": 'string',
    "City": 'string',
    "State": 'string',
    "Country": 'string'
}



Finally, we use sqlite3 to connect to our local database and create tables using our dataframes, with the correct dtypes.

In [15]:
try:
    with sqlite3.connect(r'C:\Users\meccas\Databases\CTG_Prostate.db') as conn:
        main_df.to_sql(name="CTGMain", con = conn, dtype=main_df_form, if_exists='replace', index = False)
        phases_df.to_sql(name="CTGPhases", con = conn, dtype=phases_df_form, if_exists='replace', index = False)
        interventions_df.to_sql(name="CTGInterventions", con = conn, dtype=intervention_df_form, if_exists='replace', index = False)
        organizations_df.to_sql(name="CTGOrganizations", con = conn, dtype=organization_df_form, if_exists='replace', index = False)
        collaborators_df.to_sql(name="CTGCollaborators", con= conn, dtype=collaborator_df_form, if_exists='replace', index = False)
        conditions_df.to_sql(name="CTGConditions", con = conn, dtype=conditions_df_form, if_exists='replace', index = False)
        mesh_conditions_df.to_sql(name="CTGMeshConditions", con = conn, dtype=mesh_conditions_df_form, if_exists='replace', index = False)
        mesh_interventions_df.to_sql(name="CTGMeshInterventions", con = conn, dtype=mesh_conditions_df_form, if_exists='replace', index = False)
        serious_organs_df.to_sql(name="CTGSeriousOrgans", con = conn, dtype=serious_organs_df_form, if_exists = 'replace', index = False)
        other_organs_df.to_sql(name="CTGOtherOrgans", con = conn, dtype=other_organs_df_form, if_exists = 'replace', index = False)
        locations_df.to_sql(name="CTGLocations", con = conn, dtype=locations_df_form, if_exists='replace', index = False)
        # outcomes_df.to_sql(name="CTGOutcomes", con = conn, dtype=outcomes_df_form, if_exists='replace', index=False)

except sqlite3.OperationalError as e:
    print('Failed to open database:', e)